In [1]:
import os
import json
import gzip
import subprocess
import csv
from scipy.stats import gmean


In [2]:
import os
import json
import gzip
import subprocess
import csv
from scipy.stats import gmean


dashboard = os.path.expanduser("~/code/mgs-pipeline/dashboard/")

with open(os.path.join(dashboard, "human_virus_sample_counts.json")) as inf:
    human_virus_sample_counts = json.load(inf)

with open(os.path.join(dashboard, "metadata_samples.json")) as inf:
    metadata_samples = json.load(inf)

with open(os.path.join(dashboard, "metadata_bioprojects.json")) as inf:
    metadata_bioprojects = json.load(inf)

with open(os.path.join(dashboard, "metadata_papers.json")) as inf:
    metadata_papers = json.load(inf)

studies = list(metadata_papers.keys())


def start():
    human_virus_shares = []
    zero_counts = 0

    for study in studies:
        # Dropping studies that aren't WTP based
        if study in [
            "Johnson 2023",  # unpublished data
            "Cui 2023",  # untreated undigested sludge
            "Wang 2022",  # COVID-19 hospital wastewater
            "Petersen 2015",  # air plane waste
            "Hendriksen 2019",  # man hole"
            "Moritz 2019",  # university wastewater
            "Wu 2020",  # lung sample
            "Fierer 2022",  # university campus
        ]:
            continue

        for bioproject in metadata_papers[study]["projects"]:
            samples = metadata_bioprojects[bioproject]

            if study == "Bengtsson-Palme 2016":
                samples = [
                    sample
                    for sample in samples
                    if metadata_samples[sample]["fine_location"].startswith(
                        "Inlet"
                    )
                ]

            if study == "Ng 2019":
                samples = [
                    sample
                    for sample in samples
                    if metadata_samples[sample]["fine_location"] == "Influent"
                ]

            for sample in samples:
                if metadata_samples[sample].get("enrichment") == "panel":
                    continue

                humanreads = "%s.humanviruses.tsv" % sample

                if not os.path.exists(f"humanviruses/{humanreads}"):
                    subprocess.check_call(
                        [
                            "aws",
                            "s3",
                            "cp",
                            "s3://nao-mgs/%s/humanviruses/%s"
                            % (bioproject, humanreads),
                            "humanviruses/",
                        ]
                    )

                with open(f"humanviruses/{humanreads}") as inf:
                    human_virus_reads = 0
                    for line in inf:
                        (
                            line_taxid,
                            clade_assignments,
                            _,
                        ) = line.strip().split("\t")
                        clade_hits = int(clade_assignments)
                        human_virus_reads += int(clade_hits)
                    human_virus_relative_abundance = (
                        human_virus_reads / metadata_samples[sample]["reads"]
                    )

                    # print (human_virus_relative_abundance)

                    if human_virus_relative_abundance == 0.0:
                        zero_counts += 1
                        continue

                    human_virus_shares.append(human_virus_relative_abundance)
    # Dropping all zeros from human_virus_shares

    perc_zero_human_read_samples = round(
        (zero_counts / len(human_virus_shares)) * 100, 2
    )

    gmean_viral_share = round(gmean(human_virus_shares), 7)

    return f"When dropping {zero_counts} samples without human reads ({perc_zero_human_read_samples}% of all samples), the geometric mean of samples' human read share is {gmean_viral_share * 100}% \n Put differently 1 in {round(1 / gmean_viral_share)} reads in the wastewater samples are human reads"


if __name__ == "__main__":
    print(start())

When dropping 62 samples without human reads (3.71% of all samples), the geometric mean of samples' human read share is 0.00025% 
 Put differently 1 in 400000 reads in the wastewater samples are human reads


In [3]:
def start():

    bengtsson_virus_shares = []
    ng_virus_shares = []
    
    virus_taxid = 10239

    for study in studies:
        # Dropping studies that aren't WTP based
        if study not in [
            "Bengtsson-Palme 2016", 
            "Ng 2019", ]:
            continue

        for bioproject in metadata_papers[study]["projects"]:
            samples = metadata_bioprojects[bioproject]

            if study == "Bengtsson-Palme 2016":
                samples = [
                    sample
                    for sample in samples
                    if metadata_samples[sample]["fine_location"].startswith(
                        "Inlet"
                    )
                ]

            if study == "Ng 2019":
                samples = [
                    sample
                    for sample in samples
                    if metadata_samples[sample]["fine_location"] == "Influent"
                ]
                           

            for sample in samples:
                cladecounts = "%s.tsv.gz" % sample
                with gzip.open(f"cladecounts/{cladecounts}") as inf:

                    for line in inf:
                        (
                            line_taxid,
                            _,
                            _,
                            clade_assignments,
                            _,
                        ) = line.strip().split()
                        taxid = int(line_taxid)
                        clade_hits = int(clade_assignments)
                        if taxid == virus_taxid:
                            if study == "Bengtsson-Palme 2016":
                                bengtsson_virus_shares.append(clade_hits / metadata_samples[sample]["reads"])
                            if study == "Ng 2019":
                                ng_virus_shares.append(clade_hits / metadata_samples[sample]["reads"])
                
                
            
    

    perc_gmean_bengtsson = round(gmean(bengtsson_virus_shares) * 100, 8)
    perc_gmean_ng = round(gmean(ng_virus_shares) * 100, 8)
    
    return f"The geometric mean of samples' viral read share is {perc_gmean_bengtsson}% for Bengtsson-Palme 2016 and {perc_gmean_ng}% for Ng 2019 \n In other words, 1 in {round(1 / (perc_gmean_bengtsson / 100), 2)} reads in Bengtsson-Palme 2016 and 1 in {round(1 / (perc_gmean_ng / 100), 2)} reads in Ng 2019 is a viral read."

if __name__ == "__main__":
    print(start())

The geometric mean of samples' viral read share is 0.01525929% for Bengtsson-Palme 2016 and 0.00904169% for Ng 2019 
 In other words, 1 in 6553.38 reads in Bengtsson-Palme 2016 and 1 in 11059.88 reads in Ng 2019 is a viral read.


In [4]:
def start():

    bengtsson_virus_shares = []
    yang_virus_shares = []
    
    virus_taxid = 10239

    for study in studies:
        # Dropping studies that aren't WTP based
        if study not in [
            "Bengtsson-Palme 2016", 
            "Yang 2020", ]:
            continue

        for bioproject in metadata_papers[study]["projects"]:
            samples = metadata_bioprojects[bioproject]

            if study == "Bengtsson-Palme 2016":
                samples = [
                    sample
                    for sample in samples
                    if metadata_samples[sample]["fine_location"].startswith(
                        "Inlet"
                    )
                ]


            for sample in samples:
                humanreads = "%s.humanviruses.tsv" % sample

                if not os.path.exists(f"humanviruses/{humanreads}"):
                    subprocess.check_call(
                        [
                            "aws",
                            "s3",
                            "cp",
                            "s3://nao-mgs/%s/humanviruses/%s"
                            % (bioproject, humanreads),
                            "humanviruses/",
                        ]
                    )

                with open(f"humanviruses/{humanreads}") as inf:
                    human_virus_reads = 0
                    for line in inf:
                        (
                            line_taxid,
                            clade_assignments,
                            _,
                        ) = line.strip().split("\t")
                        clade_hits = int(clade_assignments)
                        human_virus_reads += int(clade_hits)
                    human_virus_relative_abundance = (
                        human_virus_reads / metadata_samples[sample]["reads"]
                    )

                    # print (human_virus_relative_abundance)

                    if human_virus_relative_abundance == 0.0:

                        continue

                    if study == "Bengtsson-Palme 2016":
                        bengtsson_virus_shares.append(human_virus_relative_abundance)
                    if study == "Yang 2020":
                        yang_virus_shares.append(human_virus_relative_abundance)

                    
            
    

    share_gmean_bengtsson = gmean(bengtsson_virus_shares)
    share_gmean_yang = gmean(yang_virus_shares)
    
    return f"1 in {round(1 / share_gmean_bengtsson, 2)} reads in Bengtsson-Palme 2016 and 1 in {round(1 / share_gmean_yang, 2)} reads in Yang 2020 is a human-infecting virus read."


if __name__ == "__main__":
    print(start())

1 in 8324122.38 reads in Bengtsson-Palme 2016 and 1 in 1567.9 reads in Yang 2020 is a human-infecting virus read.


Creating SI table of human virus and all virus abundance.

In [6]:
def start():
    study_abundances = {}

    virus_taxid = 10239

    for study in studies:
        # Dropping studies that aren't WTP based
        if study in [
            "Johnson 2023",  # unpublished data
            "Cui 2023",  # untreated undigested sludge
            "Wang 2022",  # COVID-19 hospital wastewater
            "Petersen 2015",  # air plane waste
            "Hendriksen 2019",  # man hole
            "Moritz 2019",  # university wastewater
            "Wu 2020",  # lung sample
            "Fierer 2022",  # university campus
        ]:
            continue

       
        for bioproject in metadata_papers[study]["projects"]:
            samples = metadata_bioprojects[bioproject]

            if study == "Bengtsson-Palme 2016":
                samples = [
                    sample
                    for sample in samples
                    if metadata_samples[sample]["fine_location"].startswith(
                        "Inlet"
                    )
                ]

            if study == "Ng 2019":
                samples = [
                    sample
                    for sample in samples
                    if metadata_samples[sample]["fine_location"] == "Influent"
                ]

            
            for sample in samples:
                if metadata_samples[sample].get("enrichment") == "panel":
                    continue

    
                cladecounts = "%s.tsv.gz" % sample
                with gzip.open(f"cladecounts/{cladecounts}") as inf:
                    

                    for line in inf:
                        (
                            line_taxid,
                            _,
                            _,
                            clade_assignments,
                            _,
                        ) = line.strip().split()
                        taxid = int(line_taxid)
                        clade_hits = int(clade_assignments)
                        if taxid == virus_taxid:        
                            if study not in study_abundances:
                                study_abundances[study] = [[], []] # Initialize lists for the study if not already present
                            study_abundances[study][1].append(clade_hits /metadata_samples[sample]["reads"])
                            
            
                
                humanreads = "%s.humanviruses.tsv" % sample

                if not os.path.exists(f"humanviruses/{humanreads}"):
                    subprocess.check_call(
                        [
                            "aws",
                            "s3",
                            "cp",
                            "s3://nao-mgs/%s/humanviruses/%s"
                            % (bioproject, humanreads),
                            "humanviruses/",
                        ]
                    )

                with open(f"humanviruses/{humanreads}") as inf:
                    human_virus_reads = 0
                    for line in inf:
                        (
                            line_taxid,
                            clade_assignments,
                            _,
                        ) = line.strip().split("\t")
                        clade_hits = int(clade_assignments)
                        human_virus_reads += int(clade_hits)
                    human_virus_relative_abundance = (
                        human_virus_reads / metadata_samples[sample]["reads"]
                    )


                    if human_virus_relative_abundance > 0.0:
                        study_abundances[study][0].append(human_virus_relative_abundance) 



    # Create the table
    gmean_human_virus_shares = []
    gmean_all_virus_shares = []
    table = [("Study", "Geometric mean of human virus shares", "Geometric mean of all virus shares")]
    

    for study, abundances in study_abundances.items():
        gmean_human_virus_shares.append(gmean(abundances[0]))
        gmean_all_virus_shares.append(gmean(abundances[1]))
        table.append(
            (study,
             gmean(abundances[0]), 
             gmean(abundances[1]),
            )
        )

    table.append(("All studies", gmean(gmean_human_virus_shares), gmean(gmean_all_virus_shares)))



    return table

def format_number(num):
    ROUNDING_DIGITS = 2
    if "e" in "{:.9e}".format(num):
        base, exponent = "{:.9e}".format(num).split("e")
        rounded_base = round(float(base), ROUNDING_DIGITS)
        target_read_per_n = int(1 / num)
        return "{} * 10^{} (1 in {})".format(rounded_base, int(exponent), target_read_per_n)
    else:
        return str(num)

if __name__ == "__main__":
    table = start()


    formatted_table = []
    for row in table:
        formatted_row = []
        for cell in row:
            if isinstance(cell, float):
                formatted_row.append(format_number(cell))
            else:
                formatted_row.append(str(cell))
        formatted_table.append("\t".join(formatted_row))
    print("\n".join(formatted_table))

    

Study	Geometric mean of human virus shares	Geometric mean of all virus shares
Bengtsson-Palme 2016	1.2 * 10^-7 (1 in 8324122)	1.53 * 10^-4 (1 in 6553)
Brinch 2020	2.24 * 10^-6 (1 in 446026)	3.59 * 10^-3 (1 in 278)
Brumfield 2022	1.52 * 10^-6 (1 in 658766)	5.27 * 10^-3 (1 in 189)
Crits-Christoph 2021	5.01 * 10^-6 (1 in 199529)	5.37 * 10^-3 (1 in 186)
Maritz 2019	9.23 * 10^-7 (1 in 1083027)	8.49 * 10^-4 (1 in 1177)
McCall 2023	1.82 * 10^-6 (1 in 550960)	6.71 * 10^-4 (1 in 1489)
Munk 2022	2.73 * 10^-6 (1 in 366316)	2.42 * 10^-3 (1 in 413)
Ng 2019	4.02 * 10^-7 (1 in 2487173)	9.04 * 10^-5 (1 in 11059)
Rothman 2021	2.43 * 10^-6 (1 in 410935)	4.2 * 10^-2 (1 in 23)
Spurbeck 2023	1.36 * 10^-6 (1 in 732757)	8.38 * 10^-5 (1 in 11927)
Yang 2020	6.38 * 10^-4 (1 in 1567)	2.08 * 10^-1 (1 in 4)
All studies	2.26 * 10^-6 (1 in 441830)	1.93 * 10^-3 (1 in 516)


Creating a table, return the accessions of each study.


In [3]:

dashboard = os.path.expanduser("~/code/mgs-pipeline/dashboard/")

with open(os.path.join(dashboard, "metadata_samples.json")) as inf:
    metadata_samples = json.load(inf)

with open(os.path.join(dashboard, "metadata_bioprojects.json")) as inf:
    metadata_bioprojects = json.load(inf)

with open(os.path.join(dashboard, "metadata_papers.json")) as inf:
    metadata_papers = json.load(inf)


studies = list(metadata_papers.keys())

rows = []

for study in studies:
    if study not in [
            "Bengtsson-Palme 2016",
            "Munk 2022",
            "Brinch 2020",
            "Ng 2019",
            "Maritz 2019",
            "Brumfield 2022",
            "Rothman 2021",
            "Yang 2020",
            "Spurbeck 2023",
            "Crits-Christoph 2021",
        ]:
            continue
    bioprojects = metadata_papers[study]["projects"]
    samples = []
    for bioproject in bioprojects:
        samples = metadata_bioprojects[bioproject]
        for sample in samples:
            rows.append(
                [study, bioproject, sample])

with open("accessions.tsv", mode='w', newline='') as outf:
    writer = csv.writer(outf, delimiter='\t')
    
    # Write the header row
    writer.writerow(["Study", "Bioproject", "Sample"])
    
    # Write the data rows
    writer.writerows(rows)

Calculating the median, and upper and lower bound of RA1% of select pathogens we mention in the paper.

In [16]:
import csv
from dataclasses import dataclass

import matplotlib.pyplot as plt  # type: ignore
import numpy as np

# PERCENTILES = [2.2, 15.865, 50, 65.865, 97.8]
PERCENTILES = [5, 25, 50, 75, 95]


@dataclass
class SummaryStats:
    mean: float
    std: float
    min: float
    percentiles: dict[int, float]
    max: float

def read_data() -> dict[tuple[str, str, str, str], SummaryStats]:
    data = {}
    with open("fits_summary.tsv") as datafile:
        reader = csv.DictReader(datafile, delimiter="\t")
        for row in reader:
            virus = row["tidy_name"]
            predictor_type = row["predictor_type"]
            study = row["study"]
            location = row["location"]
            data[virus, predictor_type, study, location] = SummaryStats(
                mean=float(row["mean"]),
                std=float(row["std"]),
                min=float(row["min"]),
                percentiles={p: float(row[f"{p}%"]) for p in PERCENTILES},
                max=float(row["max"]),
            )
    return data



# Norovirus GII
# Norovirus GI
# SARS COV-2

# Polyomaviruses
# HIV

In [22]:
data = read_data()
print(data)

viruses = [
        ("Norovirus (GII)", "incidence"),
        ("Norovirus (GI)", "incidence"),
        ("SARS-COV-2", "incidence"),

        ("HIV", "prevalence"),
        ("MCV", "prevalence"),
        ("BKV", "prevalence"),
        ("JCV", "prevalence"),
]
        
studies = {
        "rothman": "Rothman",
        "crits_christoph": "Crits-Christoph",
        "spurbeck": "Spurbeck",
        "brinch": "Brinch",
    }

for virus, predictor_type in viruses:
    if predictor_type == "prevalence":
        for study in ["rothman", "crits_christoph", "spurbeck", "brinch"]:
            stats = data[virus, predictor_type, study, "Overall"]
            median = stats.percentiles[50]
            lower = stats.percentiles[5]
            upper = stats.percentiles[95]
            print(f"{virus} {study} {median} {lower} {upper}")

    else:
        for study in ["rothman", "crits_christoph", "spurbeck"]:
            stats = data[virus, predictor_type, study, "Overall"]
            median = stats.percentiles[50]
            lower = stats.percentiles[5]
            upper = stats.percentiles[95]
            print(f"{virus} {study} {median} {lower} {upper}")

{('AAV2', 'prevalence', 'brinch', 'Amager'): SummaryStats(mean=1.2671265644969398e-10, std=5.2982193733469147e-11, min=1.5829901427323194e-11, percentiles={5: 5.292968908911574e-11, 25: 8.841037231501425e-11, 50: 1.1995342415210318e-10, 75: 1.57063170853141e-10, 95: 2.2344480278304135e-10}, max=4.6397326675487977e-10), ('AAV2', 'prevalence', 'brinch', 'Avedoere'): SummaryStats(mean=4.9854400001541704e-11, std=3.142634342526885e-11, min=1.5157911890288006e-12, percentiles={5: 1.3297794599062218e-11, 25: 2.725941607840325e-11, 50: 4.36261448881117e-11, 75: 6.450754246758683e-11, 95: 1.0795798052372936e-10}, max=2.660472921877612e-10), ('AAV2', 'prevalence', 'brinch', 'Overall'): SummaryStats(mean=9.689079997562286e-11, std=2.0841168646063883e-10, min=1.0607244866568068e-12, percentiles={5: 1.4378182043072377e-11, 25: 3.6278726613889537e-11, 50: 6.035417556181804e-11, 75: 9.750617555796415e-11, 95: 2.4806074166614163e-10}, max=4.6272655940410605e-09), ('AAV2', 'prevalence', 'brinch', 'Val